In [183]:
import sympy
from sympy import Symbol

from mlscorecheck.core import (score_function_set, sqrt)
from mlscorecheck.core import fowlkes_mallows_index, markedness

In [184]:
tp = Symbol('tp')
fp = Symbol('fp')
tn = Symbol('tn')
fn = Symbol('fn')
eps = Symbol('eps')
p = Symbol('p')
n = Symbol('n')
beta = Symbol('beta')
fm = Symbol('fm')
mk = Symbol('mk')

In [185]:
tp_ = 20
tn_ = 30
p_ = 50
n_ = 100
fn_ = p_ - tp_
fp_ = n_ - tn_
fm_ = fowlkes_mallows_index(tp=tp_, fp=fp_, p=p_)
mk_ = markedness(tp=tp_, tn=tn_, fp=fp_, fn=fn_)

In [186]:
eq0 = fm - fowlkes_mallows_index(tp=tp, fp=fp, p=p)
eq1 = mk - markedness(tp=tp, tn=tn, fp=fp, fn=fn)
eq2 = p - tp - fn
eq3 = n - tn - fp

In [187]:
fp0 = sympy.solve(eq0, fp, dict=True)
fp1 = sympy.solve(eq1, fp, dict=True)
fp3 = sympy.solve(eq3, fp, dict=True)

In [188]:
print(fp0)
print(fp1)
print(fp3)

[{fp: -tp + tp**2/(fm**2*p)}]
[{fp: tp*(-fn*mk - mk*tn + tn)/(fn*mk + fn + mk*tn)}]
[{fp: n - tn}]


In [189]:
eq10 = fp0[0][fp] - fp3[0][fp]
eq11 = fp1[0][fp] - fp3[0][fp]

In [190]:
eq10

-n + tn - tp + tp**2/(fm**2*p)

In [191]:
eq11

-n + tn + tp*(-fn*mk - mk*tn + tn)/(fn*mk + fn + mk*tn)

In [192]:
tp0 = sympy.solve(eq10, tp, dict=True)
tp1 = sympy.solve(eq11, tp, dict=True)
tp2 = sympy.solve(eq2, tp, dict=True)

In [193]:
print(tp0)
print(tp1)
print(tp2)

[{tp: fm*(fm*p - sqrt(p*(fm**2*p + 4*n - 4*tn)))/2}, {tp: fm*(fm*p + sqrt(p*(fm**2*p + 4*n - 4*tn)))/2}]
[{tp: (-fn*mk*n + fn*mk*tn - fn*n + fn*tn - mk*n*tn + mk*tn**2)/(fn*mk + mk*tn - tn)}]
[{tp: -fn + p}]


In [194]:
eq20 = tp0[0][tp] - tp2[0][tp]
eq21 = tp1[0][tp] - tp2[0][tp]

In [195]:
eq20

fm*(fm*p - sqrt(p*(fm**2*p + 4*n - 4*tn)))/2 + fn - p

In [196]:
eq21

fn - p + (-fn*mk*n + fn*mk*tn - fn*n + fn*tn - mk*n*tn + mk*tn**2)/(fn*mk + mk*tn - tn)

In [197]:
tn0 = sympy.solve(eq20, tn, dict=True)
tn1 = sympy.solve(eq21, tn, dict=True)

In [198]:
tn0

[{tn: fm**2*p/4 + n - (fm**2*p + 2*fn - 2*p)**2/(4*fm**2*p)}]

In [199]:
tn1

[{tn: (-2*fn*mk + mk*n + mk*p - p - sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk)},
 {tn: (-2*fn*mk + mk*n + mk*p - p + sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk)}]

In [200]:
tn1_sol_0 = tn1[0][tn]

In [201]:
# tn1 case 0

In [202]:
eq30 = tn0[0][tn] - tn1[0][tn]

In [203]:
eq30

fm**2*p/4 + n - (-2*fn*mk + mk*n + mk*p - p - sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk) - (fm**2*p + 2*fn - 2*p)**2/(4*fm**2*p)

In [204]:
# supposing that mk > 0 and fm > 0

In [205]:
eq30 = sympy.simplify(eq30 * (2*mk*4*fm**2*p))

In [206]:
print(eq30)

4*fm**2*mk*n*p + 4*fm**2*mk*p**2 + 4*fm**2*p**2 + 4*fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) - 8*fn**2*mk + 16*fn*mk*p - 8*mk*p**2


In [207]:
expr = 4*fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2)

In [208]:
eq30_mod = sympy.simplify(sympy.Eq((eq30 - expr)**2, (-expr)**2))

In [209]:
eq30_mod = eq30_mod.lhs - eq30_mod.rhs

In [210]:
eq30_mod

-16*fm**4*p**2*(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) + 16*(fm**2*mk*n*p + fm**2*mk*p**2 + fm**2*p**2 - 2*fn**2*mk + 4*fn*mk*p - 2*mk*p**2)**2

In [211]:
fn_sol_0 = sympy.solve(eq30_mod, fn, dict=True)

In [212]:
fn_sol_0

[{fn: p},
 {fn: p - (9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)/(3*(-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(1/3)) - (-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(

In [213]:
fn_sol_0[1][fn].subs({mk: mk_, fm: fm_, p: p_, n:n_})

30.0000000000001

In [214]:
tn1_sol_0.subs({mk: mk_, fm: fm_, p: p_, n:n_, fn: 30})

240.000000000000

In [215]:
# tn1 case 1

In [216]:
tn1_sol_1 = tn1[1][tn]

In [217]:
eq30 = tn0[0][tn] - tn1[1][tn]

In [218]:
eq30

fm**2*p/4 + n - (-2*fn*mk + mk*n + mk*p - p + sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk) - (fm**2*p + 2*fn - 2*p)**2/(4*fm**2*p)

In [219]:
# supposing that mk > 0 and fm > 0

In [220]:
eq30 = sympy.simplify(eq30 * (2*mk*4*fm**2*p))

In [221]:
print(eq30)

4*fm**2*mk*n*p + 4*fm**2*mk*p**2 + 4*fm**2*p**2 - 4*fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) - 8*fn**2*mk + 16*fn*mk*p - 8*mk*p**2


In [222]:
expr = 4*fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2)

In [223]:
eq30_mod = sympy.simplify(sympy.Eq((eq30 + expr)**2, (expr)**2))

In [224]:
eq30_mod = eq30_mod.lhs - eq30_mod.rhs

In [225]:
eq30_mod

-16*fm**4*p**2*(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) + 16*(fm**2*mk*n*p + fm**2*mk*p**2 + fm**2*p**2 - 2*fn**2*mk + 4*fn*mk*p - 2*mk*p**2)**2

In [226]:
fn_sol_0 = sympy.solve(eq30_mod, fn, dict=True)

In [227]:
fn_sol_0

[{fn: p},
 {fn: p - (9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)/(3*(-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(1/3)) - (-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(

In [228]:
fn_sol_0[1][fn].subs({mk: mk_, fm: fm_, p: p_, n:n_})

30.0000000000001

In [229]:
tn1_sol_1.subs({mk: mk_, fm: fm_, p: p_, n:n_, fn: 30})

29.9999999999999

In [230]:
# the right solutions
print(fn_sol_0[1][fn])
print(tn1_sol_1)

p - (9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)/(3*(-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(1/3)) - (-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(1/3)/3
(-2*fn*mk

In [231]:
fn_sol_a = fn_sol_0[1][fn]
tn_sol_a = tn1_sol_1

In [232]:
# the other way

In [233]:
eq20 = tp0[1][tp] - tp2[0][tp]
eq21 = tp1[0][tp] - tp2[0][tp]

In [234]:
tn0 = sympy.solve(eq20, tn, dict=True)
tn1 = sympy.solve(eq21, tn, dict=True)

In [235]:
tn0

[{tn: fm**2*p/4 + n - (fm**2*p + 2*fn - 2*p)**2/(4*fm**2*p)}]

In [236]:
tn1

[{tn: (-2*fn*mk + mk*n + mk*p - p - sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk)},
 {tn: (-2*fn*mk + mk*n + mk*p - p + sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk)}]

In [237]:
# tn1 case 0

In [238]:
tn1_sol_0 = tn1[0][tn]

In [239]:
eq30 = tn0[0][tn] - tn1[0][tn]

In [240]:
eq30

fm**2*p/4 + n - (-2*fn*mk + mk*n + mk*p - p - sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk) - (fm**2*p + 2*fn - 2*p)**2/(4*fm**2*p)

In [241]:
# supposing that mk > 0 and fm > 0

In [242]:
eq30 = sympy.simplify(eq30 * (2*mk*4*fm**2*p))

In [243]:
print(eq30)

4*fm**2*mk*n*p + 4*fm**2*mk*p**2 + 4*fm**2*p**2 + 4*fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) - 8*fn**2*mk + 16*fn*mk*p - 8*mk*p**2


In [244]:
expr = 4*fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2)

In [245]:
eq30_mod = sympy.simplify(sympy.Eq((eq30 - expr)**2, (-expr)**2))

In [246]:
eq30_mod = eq30_mod.lhs - eq30_mod.rhs

In [247]:
eq30_mod

-16*fm**4*p**2*(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) + 16*(fm**2*mk*n*p + fm**2*mk*p**2 + fm**2*p**2 - 2*fn**2*mk + 4*fn*mk*p - 2*mk*p**2)**2

In [248]:
fn_sol_0 = sympy.solve(eq30_mod, fn, dict=True)

In [249]:
fn_sol_0[1][fn].subs({mk: mk_, fm: fm_, p: p_, n:n_})

30.0000000000001

In [250]:
tn1_sol_0.subs({mk: mk_, fm: fm_, p: p_, n:n_, fn: 30})

240.000000000000

In [251]:
# tn1 case 1

In [252]:
tn1_sol_0 = tn1[1][tn]

In [253]:
eq30 = tn0[0][tn] - tn1[0][tn]

In [254]:
eq30

fm**2*p/4 + n - (-2*fn*mk + mk*n + mk*p - p - sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk) - (fm**2*p + 2*fn - 2*p)**2/(4*fm**2*p)

In [255]:
# supposing that mk > 0 and fm > 0

In [256]:
eq30 = sympy.simplify(eq30 * (2*mk*4*fm**2*p))

In [257]:
print(eq30)

4*fm**2*mk*n*p + 4*fm**2*mk*p**2 + 4*fm**2*p**2 + 4*fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) - 8*fn**2*mk + 16*fn*mk*p - 8*mk*p**2


In [258]:
expr = 4*fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2)

In [259]:
eq30_mod = sympy.simplify(sympy.Eq((eq30 - expr)**2, (-expr)**2))

In [260]:
eq30_mod = eq30_mod.lhs - eq30_mod.rhs

In [261]:
eq30_mod

-16*fm**4*p**2*(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) + 16*(fm**2*mk*n*p + fm**2*mk*p**2 + fm**2*p**2 - 2*fn**2*mk + 4*fn*mk*p - 2*mk*p**2)**2

In [262]:
fn_sol_0 = sympy.solve(eq30_mod, fn, dict=True)

In [263]:
fn_sol_0[1][fn].subs({mk: mk_, fm: fm_, p: p_, n:n_})

30.0000000000001

In [264]:
tn1_sol_0.subs({mk: mk_, fm: fm_, p: p_, n:n_, fn: 30})

29.9999999999999

In [265]:
# the other true solution
print(fn_sol_0[1][fn])
print(tn1_sol_0)

p - (9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)/(3*(-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(1/3)) - (-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(1/3)/3
(-2*fn*mk

In [266]:
fn_sol_b = fn_sol_0[1][fn]
tn_sol_b = tn1_sol_0

In [267]:
fn_sol_a == fn_sol_b

True

In [268]:
tn_sol_a == tn_sol_b

True